In [2]:
import pandas as pd

df = pd.read_csv(
    'PRE_DEPARTURE_STUDENTS_MENTAL_HEALTH_SURVEY_EXTENDED.csv'
)

df.head()


,Timestamp,1. Age Group,2. Gender,3. Academic Level,Destination Country,5. How often do you feel overwhelmed by your responsibilities during pre departure preparation?,6. How often do you feel nervous or on edge about going abroad?,7. Do you find it difficult to relax when thinking about your upcoming move?,8. How often do you worry about your ability to cope with challenges abroad?,9. How often do you feel low energy or lack motivation due to pre_departure stress?,...,21. How comfortable are you discussing your worries with friends or family?,22. How concerned are you about becoming socially isolated abroad?,23. How confident are you about making new friends abroad?,24. Do you get sufficient sleep (7_8_ hours) during the preparation period?,25. How often do you engage in physical activity or exercise?,"26. Do you use relaxation techniques (prayer, meditation, deep breathing)?","27. How prepared do you feel to live independently abroad (cooking, budgeting, managing tasks)?","28. How confident are you about handling unexpected issues abroad (illness, document problems, emergencies)?",29. How strongly do you believe you can manage homesickness after moving abroad?,30. How stressed are you about leaving your family and friends?
0,2025/12/08 11:05:08 PM GMT+6,23–26,Male,Undergraduate,USA,3.0,3.0,No,4.0,5.0,...,2.0,3.0,2.0,No,1.0,No,3.0,2.0,3.0,4.0
1,2025/12/08 11:38:12 PM GMT+6,23–27,Male,Undergraduate,Cypras,3.0,3.0,Yes,2.0,3.0,...,3.0,3.0,2.0,No,2.0,No,2.0,3.0,3.0,2.0
2,2025/12/08 11:41:23 PM GMT+6,23–28,Male,Undergraduate,Australia,3.0,3.0,Yes,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025/12/08 11:43:46 PM GMT+6,23–29,Male,Undergraduate,Anywhere with a good research program,3.0,1.0,No,1.0,1.0,...,5.0,1.0,5.0,No,4.0,Yes,5.0,5.0,3.0,2.0
4,2025/12/08 11:49:38 PM GMT+6,23–30,Male,Undergraduate,USA,4.0,4.0,Yes,1.0,4.0,...,4.0,3.0,4.0,yes,4.0,Yes,3.0,4.0,4.0,3.0


In [3]:
depression_cols = [
    "5. How often do you feel overwhelmed by your responsibilities during pre departure preparation?",
    "6. How often do you feel nervous or on edge about going abroad?",
    "8. How often do you worry about your ability to cope with challenges abroad?",
    "9. How often do you feel low energy or lack motivation due to pre_departure stress?",
    "11. How anxious are you about visa delays or possible rejection?",
    "12. How often do you overthink potential problems that might happen abroad?",
    "14. How stressed do you feel about adapting to a new cultural and academic environment?",
    "20. How often do you feel pressured by your family expectations?",
    "22. How concerned are you about becoming socially isolated abroad?",
    "24. Do you get sufficient sleep (7_8_ hours) during the preparation period?",
    "30. How stressed are you about leaving your family and friends?"
]


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in depression_cols:
    df[col] = le.fit_transform(df[col].astype(str))


In [5]:
df['depression_score'] = df[depression_cols].mean(axis=1)


In [6]:
X = df.drop(columns=depression_cols + ['depression_score', 'Timestamp'])
y = df['depression_score']


In [7]:
from sklearn.impute import SimpleImputer

num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

X[num_cols] = SimpleImputer(strategy='mean').fit_transform(X[num_cols])
X[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(X[cat_cols])


In [8]:
le = LabelEncoder()
for col in cat_cols:
    X[col] = le.fit_transform(X[col])


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
import pandas as pd
import numpy as np


In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in depression_cols:
    df[col] = le.fit_transform(df[col].astype(str))


In [14]:
df['depression_score'] = df[depression_cols].mean(axis=1)


In [15]:
X = df.drop(columns=depression_cols + ['depression_score', 'Timestamp'])
y_reg = df['depression_score']   # Regression target


In [16]:
from sklearn.impute import SimpleImputer

num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

X[num_cols] = SimpleImputer(strategy='mean').fit_transform(X[num_cols])
X[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(X[cat_cols])


In [17]:
for col in cat_cols:
    X[col] = le.fit_transform(X[col])


In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    X_scaled, y_reg, test_size=0.2, random_state=42
)


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    X_scaled, y_reg, test_size=0.2, random_state=42
)


In [23]:
from sklearn.svm import SVR

svm_reg = SVR(kernel='rbf')
svm_reg.fit(X_train, y_train_reg)


SVR()

In [24]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_reg = svm_reg.predict(X_test)

print("MSE:", mean_squared_error(y_test_reg, y_pred_reg))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg))


MSE: 0.17025305332626534
R2 Score: 0.715126484730898


In [25]:
def depression_label(score):
    if score <= 1:
        return 'Low'
    elif score <= 2:
        return 'Medium'
    else:
        return 'High'

y_cls = df['depression_score'].apply(depression_label)


In [26]:
X_train, X_test, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_cls, test_size=0.2, random_state=42
)


In [27]:
from sklearn.svm import SVC

svm_clf = SVC(kernel='rbf')
svm_clf.fit(X_train, y_train_cls)


SVC()

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred_cls = svm_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test_cls, y_pred_cls))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_cls, y_pred_cls))
print("\nClassification Report:\n", classification_report(y_test_cls, y_pred_cls))


Accuracy: 0.8688524590163934

Confusion Matrix:
 [[39  0  5]
 [ 0  1  2]
 [ 1  0 13]]

Classification Report:
               precision    recall  f1-score   support

        High       0.97      0.89      0.93        44
         Low       1.00      0.33      0.50         3
      Medium       0.65      0.93      0.76        14

    accuracy                           0.87        61
   macro avg       0.88      0.72      0.73        61
weighted avg       0.90      0.87      0.87        61

